# Moving tables into FileGroups

All credit goes to the marvelous Jes - Thank you Jes [Blog Post](https://blogs.lessthandot.com/index.php/datamgmt/dbadmin/sql-server-filegroups-the-what/) 

A Notebook enabling you to see how to create file groups and move tables between them

Lets create a test database

In [ ]:
CREATE DATABASE FileGroup_Test
GO

Have a look at the filegroups

In [ ]:
USE FileGroup_Test
GO
SELECT * FROM sys.filegroups

Lets create two tables, one a heap and one with a Clustered index

In [ ]:
USE FileGroup_Test
GO
-- Thanks to Itzek for this code
 ; WITH
  L0   AS (SELECT c FROM (SELECT 1 UNION ALL SELECT 1) AS D(c)), -- 2^1
  L1   AS (SELECT 1 AS c FROM L0 AS A CROSS JOIN L0 AS B),       -- 2^2
  L2   AS (SELECT 1 AS c FROM L1 AS A CROSS JOIN L1 AS B),       -- 2^4
  L3   AS (SELECT 1 AS c FROM L2 AS A CROSS JOIN L2 AS B),       -- 2^8
  L4   AS (SELECT 1 AS c FROM L3 AS A CROSS JOIN L3 AS B),       -- 2^16
  L5   AS (SELECT 1 AS c FROM L4 AS A CROSS JOIN L4 AS B),       -- 2^32
  Nums AS (SELECT ROW_NUMBER() OVER(ORDER BY (SELECT NULL)) AS k FROM L5)

select k as StuffHere , k/2 as JunkHere,  k/4 as DuffStuffHere
 into StuffAndJunk
from nums
where k <= 1000000

 ; WITH
  L0   AS (SELECT c FROM (SELECT 1 UNION ALL SELECT 1) AS D(c)), -- 2^1
  L1   AS (SELECT 1 AS c FROM L0 AS A CROSS JOIN L0 AS B),       -- 2^2
  L2   AS (SELECT 1 AS c FROM L1 AS A CROSS JOIN L1 AS B),       -- 2^4
  L3   AS (SELECT 1 AS c FROM L2 AS A CROSS JOIN L2 AS B),       -- 2^8
  L4   AS (SELECT 1 AS c FROM L3 AS A CROSS JOIN L3 AS B),       -- 2^16
  L5   AS (SELECT 1 AS c FROM L4 AS A CROSS JOIN L4 AS B),       -- 2^32
  Nums AS (SELECT ROW_NUMBER() OVER(ORDER BY (SELECT NULL)) AS k FROM L5)

select k as StuffHere , k/2 as JunkHere,  k/4 as DuffStuffHere
 into OrderedList
from nums
where k <= 1000000

ALTER TABLE [FileGroup_Test].[dbo].[OrderedList] ALTER COLUMN StuffHere INT NOT NULL

CREATE UNIQUE CLUSTERED INDEX [OrderList_index] ON [dbo].[OrderedList] -- don't judge I cant be bothered to name things properly
([StuffHere] ASC,[JunkHere],[DuffStuffHere])


Take a look at the locations of the tables

In [ ]:
SELECT  
OBJECT_NAME(PA.object_id) AS 'Table'
, PA.object_id
, FG.name 
FROM sys.filegroups FG 
	INNER JOIN sys.allocation_units AU ON AU.data_space_id = FG.data_space_id 
	INNER JOIN sys.partitions PA ON PA.partition_id = AU.container_id 
WHERE PA.object_id = (SELECT object_id(N'dbo.StuffAndJunk'))
OR PA.object_id = (SELECT object_id(N'dbo.OrderedList'))

You can also use `sp_help` to return info

In [ ]:
exec sp_help 'dbo.OrderedList'

# Create File Groups

We can create file groups for existing database

In [ ]:
ALTER DATABASE FileGroup_Test ADD FILEGROUP FGTestFG1
ALTER DATABASE FileGroup_Test ADD FILEGROUP FGTestFG2

You can see the file groups using sys.filegroups

In [ ]:
SELECT * FROM sys.filegroups

Now we need to add some files to the file groups - Check the file paths before running this

In [ ]:
ALTER DATABASE FileGroup_Test 
ADD FILE 
(NAME = FileGroup_number1, FILENAME = 'C:\Program Files\Microsoft SQL Server\MSSQL12.MSSQLSERVERSSQL\FileGroup_FileGroup1.ndf') 
TO FILEGROUP FGTestFG1

ALTER DATABASE FileGroup_Test 
ADD FILE 
(NAME = FileGroup_number2, FILENAME = 'C:\Program Files\Microsoft SQL Server\MSSQL12.MSSQLSERVERSSQL\FileGroup_FileGroup2.ndf') 
TO FILEGROUP FGTestFG2

We can see which files are in which filegroups with 

In [ ]:
SELECT sdf.name AS [FileName],
size/128 AS [Size_in_MB],
fg.name AS [File_Group_Name]
FROM sys.database_files sdf
INNER JOIN
sys.filegroups fg
ON sdf.data_space_id=fg.data_space_id

## Change the Default file group

With new file groups we can change the default file group where objects will be created

In [ ]:
ALTER DATABASE FileGroup_Test  MODIFY FILEGROUP FGTestFG1 DEFAULT  

So when we create a new table we can see where it is created by default. You can of course specify a file group if you want

In [ ]:
 ; WITH
  L0   AS (SELECT c FROM (SELECT 1 UNION ALL SELECT 1) AS D(c)), -- 2^1
  L1   AS (SELECT 1 AS c FROM L0 AS A CROSS JOIN L0 AS B),       -- 2^2
  L2   AS (SELECT 1 AS c FROM L1 AS A CROSS JOIN L1 AS B),       -- 2^4
  L3   AS (SELECT 1 AS c FROM L2 AS A CROSS JOIN L2 AS B),       -- 2^8
  L4   AS (SELECT 1 AS c FROM L3 AS A CROSS JOIN L3 AS B),       -- 2^16
  L5   AS (SELECT 1 AS c FROM L4 AS A CROSS JOIN L4 AS B),       -- 2^32
  Nums AS (SELECT ROW_NUMBER() OVER(ORDER BY (SELECT NULL)) AS k FROM L5)

select k as StuffHere , k/2 as JunkHere,  k/4 as DuffStuffHere
 into NewStuff
from nums
where k <= 1000000

In [ ]:
SELECT  
OBJECT_NAME(PA.object_id) AS 'Table'
, PA.object_id
, FG.name 
FROM sys.filegroups FG 
	INNER JOIN sys.allocation_units AU ON AU.data_space_id = FG.data_space_id 
	INNER JOIN sys.partitions PA ON PA.partition_id = AU.container_id 
WHERE PA.object_id = (SELECT object_id(N'dbo.StuffAndJunk'))
OR PA.object_id = (SELECT object_id(N'dbo.OrderedList'))
OR PA.object_id = (SELECT object_id(N'dbo.NewStuff'))

New Stuff has been created on the default file group

# Moving Tables with clustered index to another file group


You can move a table from one filegroup to another, provided the table has a clustered index on it.

So to move dbo.OrderedList we can use this code  (You can also add ONLINE if required)

In [ ]:
USE FileGroup_Test
GO
CREATE UNIQUE CLUSTERED INDEX [OrderList_index] ON 
[dbo].[OrderedList](StuffHere, JunkHere, DuffStuffHere)
WITH DROP_EXISTING
ON FGTestFG2

And when we check, we can see that the table is on the new file group

In [ ]:
SELECT 
OBJECT_NAME(PA.object_id) AS 'Table'
, PA.object_id
, FG.name 
FROM sys.filegroups FG 
	INNER JOIN sys.allocation_units AU ON AU.data_space_id = FG.data_space_id 
	INNER JOIN sys.partitions PA ON PA.partition_id = AU.container_id 
WHERE PA.object_id = (SELECT object_id(N'dbo.StuffAndJunk'))
OR PA.object_id = (SELECT object_id(N'dbo.OrderedList'))
OR PA.object_id = (SELECT object_id(N'dbo.NewStuff'))

# Moving Tables without a clustered index (Heap) to another file group

There are two ways, neither is pretty

## Create index on another file group and drop it

Lets move StuffAndJunk from PRIMARY file group to FGTestFG1 in this way  - Beware the log file

In [ ]:
USE FileGroup_Test
GO
CREATE UNIQUE CLUSTERED INDEX [StuffAndJunk_index] ON 
[dbo].[StuffAndJunk](StuffHere, JunkHere, DuffStuffHere)

ON FGTestFG1

DROP INDEX [StuffAndJunk_index] ON 
[dbo].[StuffAndJunk]

and check

In [ ]:
SELECT 
OBJECT_NAME(PA.object_id) AS 'Table'
, PA.object_id
, FG.name 
FROM sys.filegroups FG 
	INNER JOIN sys.allocation_units AU ON AU.data_space_id = FG.data_space_id 
	INNER JOIN sys.partitions PA ON PA.partition_id = AU.container_id 
WHERE PA.object_id = (SELECT object_id(N'dbo.StuffAndJunk'))
OR PA.object_id = (SELECT object_id(N'dbo.OrderedList'))
OR PA.object_id = (SELECT object_id(N'dbo.NewStuff'))

It's on FGTestFG1

## Create new table on another file group and insert into it

Lets move StuffAndJunk from FGTestFG1 file group to FGTestFG2 in this way - Beware the log file

In [ ]:
USE FileGroup_Test
GO
CREATE TABLE [dbo].[StuffAndJunk_new](
	[StuffHere] [bigint] NULL,
	[JunkHere] [bigint] NULL,
	[DuffStuffHere] [bigint] NULL
) ON [FGTestFG2]

INSERT INTO [dbo].[StuffAndJunk_new] WITH (TABLOCK)  -- to reduce logging in simple mode_
SELECT * FROM [dbo].[StuffAndJunk]
GO

DROP TABLE [dbo].[StuffAndJunk]

EXEC sp_rename 'StuffAndJunk_new' , 'StuffAndJunk' ; 

and check

In [ ]:
SELECT 
OBJECT_NAME(PA.object_id) AS 'Table'
, PA.object_id
, FG.name 
FROM sys.filegroups FG 
	INNER JOIN sys.allocation_units AU ON AU.data_space_id = FG.data_space_id 
	INNER JOIN sys.partitions PA ON PA.partition_id = AU.container_id 
WHERE PA.object_id = (SELECT object_id(N'dbo.StuffAndJunk'))
OR PA.object_id = (SELECT object_id(N'dbo.OrderedList'))
OR PA.object_id = (SELECT object_id(N'dbo.NewStuff'))

StuffAndJunk is on FGTestFG2

# Clear up

In [ ]:
USE [master]
GO
ALTER DATABASE [FileGroup_Test] SET SINGLE_USER WITH ROLLBACK IMMEDIATE
DROP DATABASE FileGroup_Test